# Modules

In [230]:
import tensorflow as tf
import scipy
from tensorflow.python.client import device_lib
from tensorflow.keras.applications import InceptionV3, ResNet152V2
from keras.models import Sequential 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D, BatchNormalization,AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [207]:
import tensorflow as tf
print(tf.__version__)

2.10.1


In [208]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [209]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13399324727667649833
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6284115968
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6882999411273269815
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

# Dataset preparation

In [210]:
batchsize = 8

In [218]:
# Training dataset

train_data_gen = ImageDataGenerator(rotation_range=0.2, 
                                        shear_range=0.2,
                                        zoom_range=0.2, 
                                        width_shift_range=0.2,
                                        height_shift_range=0.2, 
                                        rescale=1./255,
                                        validation_split=0.2)

train_data = train_data_gen.flow_from_directory('dataset/train', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical',
                                                    subset='training'
                                                    )

validation_data = train_data_gen.flow_from_directory('dataset/train', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical',
                                                    subset='validation'
                                                    )

Found 64719 images belonging to 2 classes.
Found 16179 images belonging to 2 classes.


In [219]:
# Testing dataset

test_data_gen = ImageDataGenerator(rescale=1./255)
test_data = test_data_gen.flow_from_directory('dataset/test', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical'
                                                    )

Found 4000 images belonging to 2 classes.


# Model preparation

In [ ]:
# # Get knowledge from InceptionV3 pretrained model, so we don't need to train the model from scratch
# bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3), batch_size=batchsize))
# hmodel = bmodel.output
# hmodel = Flatten()(hmodel)

# # The only layers we need to train
# # This is the layers' illustration https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Fthe-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480&psig=AOvVaw0CYHQ2oruhWMmKK2Lxm5cN&ust=1668996857968000&source=images&cd=vfe&ved=0CBAQjRxqFwoTCLDJs9TYu_sCFQAAAAAdAAAAABAU
# hmodel = Dense(64, activation='relu')(hmodel)
# hmodel = Dropout(0.5)(hmodel)
# hmodel = Dense(2, activation='softmax')(hmodel)

# model = Model(inputs=bmodel.input, outputs=hmodel)

# for layer in bmodel.layers:
#     layer.trainable = False

# print('done')

In [241]:
# Get knowledge from InceptionV3 pretrained model, so we don't need to train the model from scratch
bmodel = ResNet152V2(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
hmodel = bmodel.output
hmodel = AveragePooling2D(pool_size = (3,3))(hmodel)
hmodel = Flatten(name= 'flatten')(hmodel)

# The only layers we need to train
# This is the layers' illustration https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Fthe-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480&psig=AOvVaw0CYHQ2oruhWMmKK2Lxm5cN&ust=1668996857968000&source=images&cd=vfe&ved=0CBAQjRxqFwoTCLDJs9TYu_sCFQAAAAAdAAAAABAU

hmodel = Dense(256, activation = "relu")(hmodel)
hmodel = Dropout(0.25)(hmodel)
hmodel = Dense(256, activation = "relu")(hmodel)
hmodel = Dropout(0.25)(hmodel)
hmodel = Dense(256, activation = "relu")(hmodel)
hmodel = Dropout(0.25)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)


for layer in bmodel.layers:
    layer.trainable = False

print('done')

done


In [242]:
model.summary()

Model: "model_36"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_43 (InputLayer)          [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 86, 86, 3)    0           ['input_43[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 40, 40, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 42, 42, 64)   0           ['conv1_conv[0][0]']             
                                                                                           

                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 20, 20, 64)   16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 20, 20, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 20, 20, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_blo

 conv3_block2_2_bn (BatchNormal  (None, 10, 10, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 10, 10, 128)  0          ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 10, 10, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_block2_out (Add)         (None, 10, 10, 512)  0           ['conv3_block1_out[0][0]',       
                                                                  'conv3_block2_3_conv[0][0]']    
          

 ization)                                                                                         
                                                                                                  
 conv3_block5_1_relu (Activatio  (None, 10, 10, 128)  0          ['conv3_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block5_2_pad (ZeroPaddin  (None, 12, 12, 128)  0          ['conv3_block5_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block5_2_conv (Conv2D)   (None, 10, 10, 128)  147456      ['conv3_block5_2_pad[0][0]']     
                                                                                                  
 conv3_blo

 conv3_block8_preact_bn (BatchN  (None, 10, 10, 512)  2048       ['conv3_block7_out[0][0]']       
 ormalization)                                                                                    
                                                                                                  
 conv3_block8_preact_relu (Acti  (None, 10, 10, 512)  0          ['conv3_block8_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv3_block8_1_conv (Conv2D)   (None, 10, 10, 128)  65536       ['conv3_block8_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv3_block8_1_bn (BatchNormal  (None, 10, 10, 128)  512        ['conv3_block8_1_conv[0][0]']    
 ization) 

...

                                                                                                  
 conv4_block29_1_bn (BatchNorma  (None, 5, 5, 256)   1024        ['conv4_block29_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block29_1_relu (Activati  (None, 5, 5, 256)   0           ['conv4_block29_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block29_2_pad (ZeroPaddi  (None, 7, 7, 256)   0           ['conv4_block29_1_relu[0][0]']   
 ng2D)                                                                                            
                                                                                                  
 conv4_blo

                                                                  'conv4_block31_3_conv[0][0]']   
                                                                                                  
 conv4_block32_preact_bn (Batch  (None, 5, 5, 1024)  4096        ['conv4_block31_out[0][0]']      
 Normalization)                                                                                   
                                                                                                  
 conv4_block32_preact_relu (Act  (None, 5, 5, 1024)  0           ['conv4_block32_preact_bn[0][0]']
 ivation)                                                                                         
                                                                                                  
 conv4_block32_1_conv (Conv2D)  (None, 5, 5, 256)    262144      ['conv4_block32_preact_relu[0][0]
                                                                 ']                               
          

                                                                                                  
 conv4_block34_2_bn (BatchNorma  (None, 5, 5, 256)   1024        ['conv4_block34_2_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block34_2_relu (Activati  (None, 5, 5, 256)   0           ['conv4_block34_2_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block34_3_conv (Conv2D)  (None, 5, 5, 1024)   263168      ['conv4_block34_2_relu[0][0]']   
                                                                                                  
 conv4_block34_out (Add)        (None, 5, 5, 1024)   0           ['conv4_block33_out[0][0]',      
          

 conv5_block1_1_conv (Conv2D)   (None, 3, 3, 512)    524288      ['conv5_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv5_block1_1_bn (BatchNormal  (None, 3, 3, 512)   2048        ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_1_relu (Activatio  (None, 3, 3, 512)   0           ['conv5_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block1_2_pad (ZeroPaddin  (None, 5, 5, 512)   0           ['conv5_block1_1_relu[0][0]']    
 g2D)     

 n)                                                                                               
                                                                                                  
 conv5_block3_3_conv (Conv2D)   (None, 3, 3, 2048)   1050624     ['conv5_block3_2_relu[0][0]']    
                                                                                                  
 conv5_block3_out (Add)         (None, 3, 3, 2048)   0           ['conv5_block2_out[0][0]',       
                                                                  'conv5_block3_3_conv[0][0]']    
                                                                                                  
 post_bn (BatchNormalization)   (None, 3, 3, 2048)   8192        ['conv5_block3_out[0][0]']       
                                                                                                  
 post_relu (Activation)         (None, 3, 3, 2048)   0           ['post_bn[0][0]']                
          

# Train the model

In [243]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [244]:
checkpoint = ModelCheckpoint('models', 
                             monitor='val_loss', 
                             save_best_only=True, 
                             verbose=3)

# patience = max num of epochs when accuracy is not increasing
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)

callbacks = [checkpoint, earlystop, learning_rate]

In [245]:
try:
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data,
          steps_per_epoch=train_data.samples//batchsize,
          validation_data=validation_data,
          validation_steps=validation_data.samples//batchsize,
          callbacks=callbacks,
          epochs=50)
except Exception as e:
    print(e)

Epoch 1/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.2241 - accuracy: 0.9105
Epoch 1: val_loss improved from inf to 0.25192, saving model to models


INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets


8089/8089 [==============================] - 411s 50ms/step - loss: 0.2241 - accuracy: 0.9105 - val_loss: 0.2519 - val_accuracy: 0.9041 - lr: 0.0010
Epoch 2/50
8089/8089 [==============================] - ETA: 0s - loss: 0.1863 - accuracy: 0.9315
Epoch 2: val_loss did not improve from 0.25192
8089/8089 [==============================] - 316s 39ms/step - loss: 0.1863 - accuracy: 0.9315 - val_loss: 0.2753 - val_accuracy: 0.8803 - lr: 0.0010
Epoch 3/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.1735 - accuracy: 0.9352
Epoch 3: val_loss did not improve from 0.25192
8089/8089 [==============================] - 329s 41ms/step - loss: 0.1735 - accuracy: 0.9352 - val_loss: 0.2999 - val_accuracy: 0.8836 - lr: 0.0010
Epoch 4/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.1689 - accuracy: 0.9392
Epoch 4: val_loss did not improve from 0.25192

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
8089/8089 [=============================

INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets


8089/8089 [==============================] - 374s 46ms/step - loss: 0.1240 - accuracy: 0.9532 - val_loss: 0.2484 - val_accuracy: 0.8996 - lr: 1.0000e-05
Epoch 9/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.1230 - accuracy: 0.9526
Epoch 9: val_loss did not improve from 0.24843
8089/8089 [==============================] - 331s 41ms/step - loss: 0.1230 - accuracy: 0.9526 - val_loss: 0.2685 - val_accuracy: 0.8998 - lr: 1.0000e-05
Epoch 10/50
8089/8089 [==============================] - ETA: 0s - loss: 0.1246 - accuracy: 0.9538
Epoch 10: val_loss did not improve from 0.24843
8089/8089 [==============================] - 335s 41ms/step - loss: 0.1246 - accuracy: 0.9538 - val_loss: 0.2530 - val_accuracy: 0.8994 - lr: 1.0000e-05
Epoch 11/50
8089/8089 [==============================] - ETA: 0s - loss: 0.1233 - accuracy: 0.9539
Epoch 11: val_loss did not improve from 0.24843

Epoch 11: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
8089/8089 [============

# Model evaluation

In [175]:
acc_training, loss_training = model.evaluate_generator(train_data)
print(acc_training)
print(loss_training)

C:\Users\igosa\AppData\Local\Temp\ipykernel_13812\2038667414.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_training, loss_training = model.evaluate_generator(train_data)


0.20669765770435333
0.9289400577545166


In [ ]:
acc_val, loss_val = model.evaluate_generator(validation_dataatadatioacc_vala)
print(acc_val)
print(loss_val)

In [ ]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)